## Dataloader

In [1]:
import os
import glob
from PIL import Image

from torch.utils.data import Dataset
import torchvision.transforms as transforms


class SurgicalVQADataset(Dataset):
    def __init__(self, seq, folder_head, folder_tail, labels, transform=None):
        
        self.transform = transform
        
        # files, question and answers
        filenames = []
        for curr_seq in seq: filenames = filenames + glob.glob(folder_head + str(curr_seq) + folder_tail)
        self.vqas = []
        for file in filenames:
            file_data = open(file, "r")
            lines = [line.strip("\n") for line in file_data if line != "\n"]
            file_data.close()
            for line in lines: self.vqas.append([file, line])
        print('Total files: %d | Total question: %.d' %(len(filenames), len(self.vqas)))
        
        # Labels
        self.labels = labels
        
    def __len__(self):
        return len(self.vqas)

    def __getitem__(self, idx):
        
        # img
        loc = self.vqas[idx][0].split('/')
        img_loc = os.path.join(loc[0],loc[1],loc[2], 'left_frames',loc[-1].split('_')[0]+'.png')
        img = Image.open(img_loc)
        if self.transform: img = self.transform(img)
            
        # question and answer
        question = self.vqas[idx][1].split('|')[0]
        label = self.labels.index(str(self.vqas[idx][1].split('|')[1]))

        return img, question, label

## Model

In [2]:
from sentence_transformers import SentenceTransformer
import torch
import torch.nn as nn
from torchvision import models

class Surgical_VQA(nn.Module):
    def __init__(self, num_classes=12):
        super(Surgical_VQA, self).__init__()

        # text processing
        self.text_feature_extractor = SentenceTransformer('bert-base-nli-mean-tokens')
        # image processing
        self.img_feature_extractor = models.resnet50(pretrained=True)
        new_fc = nn.Sequential(*list(self.img_feature_extractor.fc.children())[:-1])
        self.img_feature_extractor.fc = new_fc

        #classifier
        self.classifier = nn.Linear(2816, num_classes)

    def forward(self, img, text):
        img_feature = self.img_feature_extractor(img)
        
        text_feature = self.text_feature_extractor.encode(text)
        text_feature = torch.tensor(text_feature).cuda()
        
        img_text_features = torch.cat((img_feature, text_feature), dim=1)
        
        out = self.classifier(img_text_features)
        return out

## Test model

In [3]:
def test_model(epoch, model, valid_dataloader):
    
    model.eval()
    
    total = 0
    correct = 0
    total_loss = 0.0
    
    criterion = nn.CrossEntropyLoss()
    
    with torch.no_grad():
        for i, (imgs, q, labels) in enumerate(valid_dataloader, 0):
            questions = []
            for question in q: questions.append(question)
            imgs, labels = imgs.cuda(), labels.cuda()
            
            outputs = model(imgs, questions)
            
            loss = criterion(outputs,labels)
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    # loss and acc
    total_loss = total_loss / len(valid_dataloader)
    test_acc = (correct/total)
    
    print('Test: epoch: %d loss: %.6f | Acc: %.6f' %(epoch, total_loss, test_acc))
    
    return (test_acc)

## Train model

In [4]:
from torch import optim
def train_model(epoch, model, train_dataloader, lr):  # train model
    
    model.train()
    
    total = 0
    correct = 0
    total_loss = 0.0
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr = lr, weight_decay = 0)
    
    for i, (imgs, q, labels) in enumerate(train_dataloader,0):
        questions = []
        for question in q: questions.append(question)
        imgs, labels = imgs.cuda(), labels.cuda()
        
        # zero the parameter gradients
        optimizer.zero_grad()

        outputs = model(imgs, questions)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    # loss and acc
    total_loss = total_loss / len(train_dataloader)
    train_acc = (correct/total)

    print('Train: epoch: %d loss: %.6f | Acc: %.6f' %(epoch, total_loss, train_acc))
    
    return

## Main

In [ ]:
import os
import torch

from torchvision import transforms
from torch.utils.data import DataLoader

os.environ["CUDA_VISIBLE_DEVICES"]="0"

def seed_everything(seed=27):
    '''
    Set random seed for reproducible experiments
    Inputs: seed number 
    '''
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
if __name__ == "__main__":
     
    # Set random seed
    seed_everything()  
    
    # Device Count
    num_gpu = torch.cuda.device_count()
    
    # hyperparameters
    bs = 32
    epochs = 250
    lr = 0.00001
    
    checkpoint_dir = 'checkpoints/v1/simple/'
    
    # train and test dataloader
    train_seq = [2, 3, 4, 6, 7, 9, 10, 11, 12, 14, 15]
    val_seq = [1, 5, 16]
    folder_head = 'dataset/instruments18/seq_'
    folder_tail = '/vqa/simple/*.txt'

    labels = ['kidney',
          'Idle', 'Grasping', 'Retraction', 'Tissue_Manipulation',
          'Tool_Manipulation', 'Cutting', 'Cauterization', 'Suction', 
          'Looping', 'Suturing', 'Clipping', 'Staple', 'Ultrasound_Sensing',
          'left-top', 'right-top', 'left-bottom', 'right-bottom']

    transform = transforms.Compose([
                transforms.Resize((300,256)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
                ])

    # train_dataset
    train_dataset = SurgicalVQADataset(train_seq, folder_head, folder_tail, labels, transform=transform)
    train_dataloader = DataLoader(dataset=train_dataset, batch_size= bs, shuffle=True)

    # Val_dataset
    val_dataset = SurgicalVQADataset(val_seq, folder_head, folder_tail, labels, transform=transform)
    val_dataloader = DataLoader(dataset=val_dataset, batch_size= bs, shuffle=False)
    
    # model
    model = Surgical_VQA(num_classes=len(labels)).cuda()
    
    best_epoch = [0]
    best_results = [0.0]
    
    for epoch in range(1, epochs):
        train_model(epoch, model, train_dataloader, lr)
        test_acc = test_model(epoch, model, train_dataloader)
    
        if test_acc >= best_results[0]:
            best_results[0] = test_acc
            best_epoch[0] = epoch
        
        print('Best epoch: %d | Best acc: %.6f' %(best_epoch[0], best_results[0]))
        checkpoint = {'lr': lr, 'b_s': bs, 'state_dict': model.state_dict() }
        save_name = "checkpoint_" + str(epoch) + '_epoch.pth'
        
        torch.save(checkpoint, os.path.join(checkpoint_dir, save_name))

Total files: 1560 | Total question: 9014
Total files: 447 | Total question: 2769
Train: epoch: 1 loss: 2.195718 | Acc: 0.304859
Test: epoch: 1 loss: 1.791958 | Acc: 0.493011
Best epoch: 1 | Best acc: 0.493011
Train: epoch: 2 loss: 1.667827 | Acc: 0.523297
